# Investigating Observed Patterns of Ocean Temperature, Salinity, and Sea Level
The goal of this notebook is to investigate observed patterns of ocean temperature, salinity, and sea level. An in-depth analysis is done to better conceptualize these variables and put them into context of how they affect the climate. Observed ocean temperature and salinity data from World Ocean Atlas (WOA18) is compared to climate model output from GDFL- ESM4's historical experiment to assess model representation of ocean temperatures and salinity. This analysis was conducted by Grace Woolslayer (grace.woolslayer@temple.edu), Tyler Wassel (tyler.wassel@temple.edu), and Ryan Pszczolkowski (ryan.pszczolkowski@temple.edu).  

# Background

Before we proceed with the analysis, it's crucial to have a baseline understanding of how temperature and salinity can affect sea level. Below is a detailed explanation of each component and how they affect the broader climate system.

## Ocean Heat Content

Our planet is often referred to as the blue planet because over 70% of it is covered by water. It is where life first emerged and evolved into what it is today. Water has many unique properties that have allowed it to sustain life directly and indirectly. The ocean, more specifically, plays a vital role in mitigating the effects of climate change by slowing down the rate at which our planet warms. It can store excess heat due to its high specific heat capacity, depth, and circulation patterns. A high specific heat capacity is significant because it takes much more energy (heat) to change temperature by one degree Celsius.  

The heat that is stored in the ocean is referred to as ocean heat content. The upper layers of the ocean experience greater accumulations of heat than the deeper parts of the water column but both have seen an increase in heat gain rates in the last few decades.  Ocean waves, tides, and currents in addition to large-scale circulation patterns help distribute heat from the surface to deeper, cooler levels. A rise in ocean heat content is a result of more heat being absorbed than released.  

The heat that is absorbed will eventually find its way back into Earth’s system by melting ice shelves or reheating the atmosphere. Since water has a high specific heat and most of the heat, it can take decades until the effects of the absorbed heat are seen.  

In terms of how ocean heat is measured, scientists rely on sea surface heights to calculate heat content since water expands when it’s warmed. This concept is referred to as thermal expansion. To gain insights into ocean heat content at different depths, there are other methods similar to the ones mentioned above. Scientists can use CTDs, Argo floats, and even employ seals to capture temperatures at hard-to-get depths.  

Heat content is commonly expressed in units of joules and anomalies are expressed in watts per square meter. The change in heat content is relatively small per square meter but the whole ocean must be considered, which is over 360 million square kilometers. Given that the heat content in the ocean will eventually be released, it is crucial to understand how much heat is being stored, how it is being transported, and how long it will take to be released. Heat content does not only affect sea levels but also the biochemistry of the ocean. 

To recap, increased heat in the ocean causes water to expand and decreases the solubility capacity for gases. Warm water cannot absorb as much gas, in this case carbon dioxide, as cold water. This also poses issues for global temperatures since that means that more carbon dioxide will reside in the atmosphere because of the decreased capacity of the ocean. More carbon dioxide in the atmosphere means higher global temperatures and the cycle continues.  

To learn more, visit https://www.climate.gov/news-features/understanding-climate/climate-change-ocean-heat-content

![](https://upload.wikimedia.org/wikipedia/commons/a/ac/Thermal_expansion_cartoon.jpg)

*image from [wiki media](https://commons.wikimedia.org/wiki/File:Thermal_expansion_cartoon.jpg)*

## Steric Sea Level

It is a common misconception that sea level is only changing due to melting glaciers, but several factors beyond this component contribute to sea level rise. The rise in sea level due to the addition of water is referred to as the eustatic change [Chan](https://www.esi.utexas.edu/files/047-Learning-Module-Expansion.pdf). In this analysis, we will focus on the less well-known science behind sea level rise called steric change. Steric change is the expansion of water molecules. The word steric itself is a chemistry term that refers to the spatial arrangement of atoms in a molecule. Steric sea level or steric height is the combined variation of ocean volume due to changes in salinity and temperature. Currently, the average rate of change is 1.3 mm/yr [(NASA)](https://sealevel.nasa.gov/understanding-sea-level/key-indicators/steric-height/).

![](https://lydiacneal.wordpress.com/wp-content/uploads/2015/09/k1_g_wasser-molekuele_e_en.jpg)

*image from [E-SEA](https://lydiacneal.wordpress.com/2015/10/14/well-hot-salt-ocean-power/)*

### Thermosteric Sea Level

The thermosteric sea level change is due to a process mentioned earlier called thermal expansion. If you already forgot, let me jog your memory. Thermal expansion describes the increase in volume of a substance due to the absorption of heat. Heat excites molecules which causes them to have higher energy and causes them to bump into each other more frequently, taking up more space. The increase in volume results in a lower density. The reverse process happens when water is cooled. Instead of expanding, water will contract and take up less space, causing an increase in density and a decrease in volume. Although the melting of glaciers can increase sea level, they also act to cool the ocean and counteract thermal expansion [PSU](https://www.e-education.psu.edu/earth107/node/1502).  

### Halosteric Sea Level

Thermosteric sea level change is widely accepted as being more influential on sea level rise but there is a density-driven aspect to be considered as well. Halosteric sea level is associated with the change in seawater salinity. Regional salinity-driven sea level trends have been less studied than the temperature ones since there generally has been a lack of historical measurements and because the contribution to mean sea level is close to zero [(Llovel, W.,Lee, T., 2015)](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2014GL062611). 

Saltwater is more dense than freshwater, meaning it has less volume. In other words, it takes up less space. Density and volume are inversely proportional so when one increases, the other will decrease. As more ice melts, the freshwater runoff into the ocean causes a decrease in density. This not only causes issues for sea level rise but also global ocean circulation patterns.  

![](https://mynasadata.larc.nasa.gov/sites/default/files/inline-images/pasted%20image%200_7.png)

*image from [my NASA data](https://mynasadata.larc.nasa.gov/basic-page/ocean-circulation-patterns)*

---

## Importing all Necessary Packages and Masks

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
%matplotlib inline
%config InlineBackend.figure_format='retina'
import cmocean
import cartopy.crs as ccrs
import matplotlib.path as mpath
import nc_time_axis
import pandas as pd
import matplotlib as mpl
import momlevel as ml
import gcsfs
import zarr
from matplotlib.ticker import FormatStrFormatter
import cftime
import datetime
import dask

## Importing the Data

### WOA18 

In [1]:
#insert WOA data here

Ocean temperature and salinity data is from World Ocean Atlas (WOA). “The World Ocean Atlas (WOA) is a collection of objectively analyzed, quality-controlled temperature, salinity, oxygen, phosphate, silicate, and nitrate means based on profile data from the World Ocean Database (WOD). It can be used to create boundary and/or initial conditions for various ocean models, verify numerical simulations of the ocean, and corroborate satellite data”.  

“The World Ocean Database (WOD) is the world's largest collection of uniformly formatted, quality controlled, publicly available ocean profile data. It is a powerful tool for oceanographic, climatic, and environmental research, and the end result of more than 20 years of coordinated efforts to incorporate data from institutions, agencies, individual researchers, and data recovery initiatives into a single database. WOD data spans from Captain Cook's 1772 voyage to the contemporary Argo period, making it a valuable resource for long-term and historical ocean climate analysis. Original versions of the 20,000+ datasets in the WOD are available through the NCEI archives” [(NOAA)](https://www.ncei.noaa.gov/products/world-ocean-database).   

Ocean temperature is measured in degrees Celsius and salinity is unitless. Data can be accessed through this link [https://www.ncei.noaa.gov/access/world-ocean-atlas-2018/]()

### Ocean heat content, steric sea level rise, thermosteric sea level rise, and halosteric sea level rise anomaly data

In [2]:
#insert anomaly data here

Anomaly observations provide insight for depths 0-2000 meters. Anomalies are computed over the top 700 meters (0-700) and the upper 2000 meters (0-2000). Data collected from 1955 – present. [here](https://www.ncei.noaa.gov/access/global-ocean-heat-content/index.html)

### How is Observed Data Collected?
Temperature and salinity data can be collected throughout different depths of the water column through electrical sensors. The most common method is deploying a sensor referred to as a CTD which stands for conductivity, temperature, and depth. Depth is derived through pressure sensors and salinity can be derived from conductivity which is how well a solution can conduct electricity [NOAA](https://oceanexplorer.noaa.gov/explainers/data.html#:~:text=Two%20types%20of%20data%20are,similar%20to%20elevation%20on%20land).   

An additional way that scientists gather information on temperature and salinity is using Argo floats. These floats free-floatingting in ue supper 2,000 meters of the ocean. They’re able to provide valuable information about the conditions of the ocean while being fully autonomous.  

### Model Data

In [3]:
#insert model data for temp

In [4]:
#insert model data for salinity

Monthly simulated ocean temperatures and sea water salinity output from GFDL-ESM4 under the historical experiment spanning from 1980 – 2014 was developed at NOAA's Geophysical Fluid Dynamics Laboratory (GFDL). This data contributed to the sixth phase of the Coupled Model Intercomparison Project (CMIP6) and was made available via Google Cloud [https://console.cloud.google.com/marketplace/product/noaa-public/cmip6.](https://console.cloud.google.com/marketplace/product/noaa-public/cmip6.)

---

# Analysis
Now that we've established a basic understanding of ocean heat content, steric, halosteric, and thermosteric sea level, let's dive into the analysis!

## Observed Ocean Heat Content

In [ ]:
#0-700m

In [1]:
#0-2000m

## Observed Ocean Temperatures

### Sea Surface Temperatures 

### Ocean Temperatures at 1000 Meters

## ESM4 Historical Model Ocean Temperatures 

In [2]:
#surface temp

In [3]:
# at depth 1000m

## Observed Steric Sea Level

In [4]:
# steric

In [5]:
#halosteric

In [6]:
#thermosteric

## ESM4 Model Sea Salinity

In [2]:
google_cloud_cmip6 = pd.read_csv('https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv')

In [3]:
gfdl_esm4_so_data = google_cloud_cmip6.query(
    "activity_id=='CMIP' & \
    table_id == 'Omon' & \
    variable_id == 'so' & \
    experiment_id == 'historical' & \
    institution_id == 'NOAA-GFDL' &  \
    source_id == 'GFDL-ESM4' & \
    grid_label == 'gr' & \
    member_id == 'r1i1p1f1'"
)

In [4]:
gfdl_esm4_so_data

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
245217,CMIP,NOAA-GFDL,GFDL-ESM4,historical,r1i1p1f1,Omon,so,gr,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/hist...,NaN,20190726


In [6]:
gcs = gcsfs.GCSFileSystem(token='anon')
zstore = gfdl_esm4_so_data.zstore.values[-1]
mapper = gcs.get_mapper(zstore)

gfdl_esm4_so = xr.open_zarr(mapper, consolidated=True)
gfdl_esm4_so

<xarray.Dataset>
Dimensions:    (lat: 180, bnds: 2, lev: 35, lon: 360, time: 1980)
Coordinates:
  * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lev        (lev) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
    lev_bnds   (lev, bnds) float64 dask.array<chunksize=(35, 2), meta=np.ndarray>
  * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(360, 2), meta=np.ndarray>
  * time       (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    so         (time, lev, lat, lon) float32 dask.array<chunksize=(12, 35, 180, 360), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36500.0
    comment:                <null ref>
    ...                     ...
    variable_id:            so
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/e61aa2fb-e885-4212-9284-9a77ba0f55e5...
    version_id:             v20190726

---

# Model Assessment

# The Big Picture

Changes in ocean heat content and salinity not only have implications for sea level rise but also play a crucial role in the overall climate system by driving ocean circulation. Other factors affect circulation such as surface winds but thermohaline circulation is more important for the movement of water in the deep ocean [NOAA](https://oceanservice.noaa.gov/education/tutorial_currents/05conveyor1.html#:~:text=As%20the%20seawater%20gets%20saltier,driving%20the%20global%20conveyer%20belt.). Deep ocean currents are driven by salinity and temperature gradients. Thermohaline circulation begins in the polar regions where water is the coldest and densest. Sea ice formation only occurs in the polar regions and when it does, brine rejection occurs causing the surrounding water to become saltier. As you learned, cold, salty water sinks which facilitates the conveyor belt of circulation. Ocean circulation is important since it allows for the movement and storage of heat, the distribution of nutrients, and has a strong influence on regional weather patterns. Alterations in ocean circulation can have adverse effects on all those processes [NOAA](
https://oceanexplorer.noaa.gov/facts/climate.html#:~:text=Ocean%20currents%20act%20much%20like,solar%20radiation%20reaching%20Earth's%20surfac). . If a climate denier at family dinner doesn’t believe in the effects of climate change or understands the scientific aspect of it all, talking about the financial consequences might put it into perspective for them.

Sea level rise is one of the most destructive side effects of climate change [(Yue, C., et al., 2023)](https://www.nature.com/articles/s41612-023-00466-4). It is projected to directly affect more than 1 billion people living in low-elevation coastal zones by 2100 under the SSP5.8-5 scenario. Sea level rise has already been decreasing property value and causing costly changes to infrastructure. This is particularly evident within the insurance industry, especially in Florida. There is currently a homeowners insurance crisis happening in Florida since many companies are leaving the state, liquidating, or refusing to insure policies in high-risk areas [(Duvall)](https://www.insurance.com/home-and-renters-insurance/home-insurers-leaving-florida). Some of the largest insurance companies such as Progressive and AAA are leaving or reducing their coverage in Florida. However, Nationwide and State Farm are still doing normal business in Florida. Despite the crises, six new insurance companies were approved in 2023 to issue policies. Experts in the field have analyzed the situation and attributed the retreat to rising litigation costs, the pullback of reinsurers, and Florida’s market heavily relying on small insurers that ultimately rely on reinsurers and a booming housing market along the coast. However, those coastal communities are being and will continue to be victims of intense flooding and increasingly strong hurricanes. Floods and hurricanes take an unbearable financial toll on insurance companies that many are not prepared for without raising already costly premium prices. Florida isn’t the only state that insurance companies are pulling out of, California is facing a similar crisis due to increased wildfires. Staying on topic though, other coastal communities are likely already facing similar challenges in finding affordable protection for their homes as sea level continues to rise and hurricanes become stronger due to increased sea surface temperatures.    

![](https://upload.wikimedia.org/wikipedia/commons/4/4c/Thermohaline_Circulation_2.png)

*image from [Wikipedia](https://en.wikipedia.org/wiki/Thermohaline_circulation)*

---

# Author Contribution 

Grace led the conceptual side of the analysis by researching ocean heat content, steric, thermosteric, and halosteric sea level. She also formulated the final group notebook template to create a cohesive, organized flow of information. She proofread the blog post and the final notebook, ensuring both were ready for the final submission. She made edits when necessary.